**Pipeline:**  

import spreadsheet -> preprocess matches (columns in toydata) ->  
create elo class -> add players for teams-> init team dicts -> bymatch: expectation, gameOver, update dicts/rating ->  
massage final results into output format -> save output -> websitey stuff

In [1]:
#imports
import openpyxl
import pandas as pd
import requests
from io import BytesIO

In [2]:
# Read in toy data spreadsheet tabs as separate dataframes in a list, season_data
spreadsheetId = "1mH_HUNLF_hj2lekOxKMOKM7RiTko4HvjdJryC5M69bM" # Please set your Spreadsheet ID.
url = "https://docs.google.com/spreadsheets/export?exportFormat=xlsx&id=" + spreadsheetId
res = requests.get(url)
data = BytesIO(res.content)
xlsx = openpyxl.load_workbook(filename=data)
season_data = []
for name in xlsx.sheetnames:
    new_df = pd.read_excel(data, sheet_name=name)
    new_df.name = name
    season_data.append(new_df)

# Need to remove some rows in the Match Records, filter via rows where matches are played
matches = season_data[0]
matches = matches[matches['Played?'] == True]
season_data[0] = matches

In [4]:
matches.head()

,Team 1,Team 2,Week,Played?,Map 1,Winner,Map 2,Winner.1,Map 3,Winner.2,Map 4,Winner.3,Map 5,Winner.4,Match Winner
0,A,B,1.0,True,Lijiang Tower,A,Dorado,A,King's Row,B,Hanamura,A,NaN,NaN,A
1,C,D,1.0,True,Lijiang Tower,D,Havana,D,Eichenwalde,C,Temple of Anubis,C,Busan,D,D
2,E,F,1.0,True,Lijiang Tower,E,Dorado,E,King's Row,E,NaN,NaN,NaN,NaN,E
3,G,H,1.0,True,Lijiang Tower,H,Havana,G,King's Row,H,Temple of Anubis,H,NaN,NaN,H
4,I,J,1.0,True,Lijiang Tower,I,Dorado,J,Eichenwalde,J,Hanamura,J,NaN,NaN,J


In [38]:
# Define ELO model
K = 125 # Update scaler
Rm = 3200 # mean elo initialization
delta = 500

class Team:
    def __init__(self, name, Rm=3200):
        self.currentRating = Rm
        self.opponentHist = []
        self.ratingHist = []
        self.currentRanking = 0
        
class Elo:
    def __init__(self,Rm = 3200, k=125, g=1, delta=500):
        self.ratingDict = {}
        self.teamsDict = {}
        self.k = k
        self.g = g
        self.rankings = {}

    def addPlayer(self,name, Rm = 3200):
        self.ratingDict[name] = Rm # update code by deleting this and subsuming into teams dict
        self.teamsDict[name] = Team(name, Rm)

    def gameOver(self, winner, loser):
        # take single game, chop up number of maps & winner of each
        # combine all updates at once
        #   expectation =  expected score | number of maps & team elos
        #   diff = total map wins (+.5 for draws) - expectation
        result = self.expectResult(self.ratingDict[winner], self.ratingDict[loser]) # compute the expected result
        
        # redundancy
        self.ratingDict[winner] = self.ratingDict[winner] + (self.k*self.g)*(1 - result)  # update elo rating based on error
        self.ratingDict[loser] 	= self.ratingDict[loser] + (self.k*self.g)*(0 - (1 -result))
        
        self.teamsDict[winner].currentRating =  self.ratingDict[winner] + (self.k*self.g)*(1 - result)
        self.teamsDict[loser].currentRating = self.ratingDict[loser] + (self.k*self.g)*(0 - (1 -result))  
                                                                            
        # update the histories
        self.teamsDict[winner].opponentHist.append(loser)
        self.teamsDict[winner].ratingHist.append(self.teamsDict[winner].currentRating)
        self.teamsDict[loser].opponentHist.append(winner)
        self.teamsDict[loser].ratingHist.append(self.teamsDict[loser].currentRating)
                                                                            

    def expectResult(self, p1, p2):
        exp = (p2-p1)/delta
        return 1/((10.0**(exp))+1)

In [19]:
# initialize the elo model
model = Elo()

# initialize teams
players = ['A','B','C','D','E','F','G','H','I','J']
for player in players:
    model.addPlayer(player)
#

In [27]:
winner = 'A'
model.teamsDict[winner].currentRating += 1000
model.teamsDict[winner].currentRating


2000

In [48]:
# initialize the elo model
GWL = Elo()

# initialize teams
teams = ['A','B','C','D','E','F','G','H','I','J']
for team in teams:
    GWL.addPlayer(team)

for i, game in matches.iterrows():
    if i == 0:
        # Identify winner and loser
        winner = game['Match Winner']
        if game['Team 1'] == winner:
            loser = game['Team 2']
        else:
             loser = game['Team 1']
        # update elo
        GWL.gameOver(winner,loser)
vars(GWL.teamsDict['A'])

{'currentRating': 3325.0,
 'opponentHist': ['B'],
 'ratingHist': [3325.0],
 'currentRanking': 0}

In [ ]:
# NEXT STEP: MASSAGE OUTPUT INTO FORMAT WE WANT

In [11]:
from operator 		import itemgetter
from collections 	import defaultdict
import scipy.stats 	as st
import numpy 		as np
import pandas		as pd

def k_factor(margin_of_victory, elo_diff):
    init_k	=	20
    if margin_of_victory>0:
        multiplier	=	(margin_of_victory+3) ** (0.8) / (7.5 + 0.006 * (elo_diff))
    else:
        multiplier	=	(-margin_of_victory+3)** (0.8) / (7.5 + 0.006 *(-elo_diff))
    return init_k*multiplier,init_k*multiplier

def s_value(home_score, away_score):
    S_home,S_away=0,0
    if home_score > away_score:
        S_home = 1
    elif away_score > home_score:
        S_away = 1
    else:
        S_home,S_away=.5,.5
    return S_home,S_away

def elo_update(home_score, away_score, home_rating,away_rating, home_advantage = 100.):
    home_rating     +=	home_advantage
    elo_home 	    = 	elo_prediction(home_rating,away_rating)
    elo_away        =   1 			- elo_home
    elo_diff		=	home_rating	- away_rating
    MOV				=	home_score	- away_score
    
    s_home,s_away = s_value(home_score,away_score)
    if s_home>0:
        K_home,K_away =  k_factor(MOV,elo_diff)
    else:
        K_home,K_away =  k_factor(MOV,elo_diff)
        
    return K_home*(s_home-elo_home),K_away*(s_away-elo_away)

def elo_prediction(home_rating,away_rating):
    return 1./(1 + 10 ** ((away_rating - home_rating) / (400.)))
    

def score_prediction(home_rating,away_rating):
    return (home_rating-away_rating)/28.
	


		
class EloSimulation(object):
    def __init__(self, games, update_function, label_dict, end_of_season_correction, prediction_function=None,):
        self.update_function			= update_function
        self.games						= games
        self.ratings					= {}
        self.prediction_function 		= prediction_function
        self.predictions 				= []
        self.curr_season				= defaultdict(lambda: self.games[0][1][label_dict['year_id']])
        self.end_of_season_correction 	= end_of_season_correction

    def train(self):
        for idx, game in self.games:
            new_year	= game[label_dict['year_id']]
            label_i		= game[label_dict['fran_id']]
            label_j		= game[label_dict['opp_fran']]
            
            if self.ratings.get(label_i, False ) == False:
                self.ratings[label_i] 		= elo_lookup(label_i,game[label_dict['gameorder']])
            
            if self.ratings.get(label_j,False )== False:
                self.ratings[label_j] 		= elo_lookup(label_j,game[label_dict['gameorder']])
                
            if self.curr_season[label_i]!=new_year:
                self.curr_season[label_i]=new_year
                self.ratings[label_i]=self.ratings[label_i]*.6+1505.*.4
            elif self.curr_season[label_j]!=new_year:
                self.curr_season[label_j]=new_year
                self.ratings[label_j]=self.ratings[label_j]*.6+1505.*.4
            self.predictions.append(elo_prediction(self.ratings[label_i]+100, self.ratings[label_j]))
            #todo change below to just use event
            update=self.update_function(game[label_dict['pts']],game[label_dict['opp_pts']], self.ratings[label_i], self.ratings[label_j])
            self.ratings[label_i]+=update[0]
            self.ratings[label_j]+=update[1]
            
            

    def power_rankings(self):
        
        power_rankings 	= sorted(self.ratings.items(), key=itemgetter(1), reverse=True)
        power 			= []
        for i, x in enumerate(power_rankings):
            power.append((i + 1, x))
        return power

label_dict = {
	'year_id'	:'year_id',
	'fran_id'	:'fran_id',
	'opp_fran'	:'opp_fran',
	'gameorder'	:'gameorder',
	'pts'		:"pts",
	'opp_pts'	:"opp_pts"

}
full_df = pd.read_csv("../../elo-simulations/nbaallelo.csv")
games=full_df[full_df['game_location']=='H'] #remove duplicated rows work with our elo implementation
games['SEASON']=games['year_id'].apply(lambda x: "%s-%s"%(x-1,x))

STARTING_LOC=0
def elo_lookup(fran_id,gameorder):
    return full_df[(full_df['fran_id']==fran_id)&(full_df['gameorder']>=gameorder)]['elo_i'].iloc[0]

m =  			EloSimulation(
				 games 						= list(games[games['gameorder']>STARTING_LOC].iterrows()),
				 update_function 			= elo_update, 
				 prediction_function 		= elo_prediction,
				 label_dict					= label_dict,
				 end_of_season_correction  	= 1)

m.train()
m.power_rankings()
games['prediction']=m.predictions
games['predictedWinner']=games['prediction'].apply(lambda x: 1 if x>=.5 else 0)
games['winner']=games.apply(lambda x: x['pts']>=x['opp_pts'],axis=1)

from sklearn.metrics import confusion_matrix
conf_matrix=confusion_matrix(games['winner'],games['predictedWinner'])

top = float(conf_matrix[0][0]+conf_matrix[1][1])
botton = top + float(conf_matrix[0][1] + conf_matrix[1][0])

print(top/botton * 100)


ImportError: attempted relative import with no known parent package

Output: json file with:
{
teams = list({name: Az Ots, curr_elo: 1023, current_ranking: 3, history_elo: [], history_opponents: []},
{name: P Pusheens, curr_elo: 1023, current_ranking: 3, history_elo: [], history_opponents: []},...)
}
- power rankings: [Team1, team2]
- current elo
- history of elos (by week)
- history of opponents??